In [1]:
!pip install fpdf geopandas


In [2]:
!pip install -U kaleido

# Data Engineering

In [ ]:
import os
import shutil
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objs as go
from plotly.offline import iplot
import plotly.io as pio

# ---------------------------------------------------
# 1. Load Data
print("Loading data from server...")
columns = [
    "price", "Date_of_Transfer", "postcode", 
    "Property_Type", "Old/New", "Duration", "PAON", "SAON", "Street", 
    "Locality", "Town/City", "District", "County", "PPDCategory_Type", 
    "Record_Status - monthly_file_only"
]

# Build the CSV file path from the temporary directory
csv_file_path = os.path.join(TEMP_DATASET_DIR, "pp-complete.csv")

# Load only Date_of_Transfer to save memory
df = pd.read_csv(
    csv_file_path,
    header=None, 
    names=columns, 
    parse_dates=["Date_of_Transfer"],
    dayfirst=False
)
print("Data loaded. Total records:", len(df))
print("Sample data:")
print(df.head())


# --------------------------------------------------------------------
# macbook code starts
# -
# Exclude unwanted columns
columns_to_exclude = [
    'Transaction_unique_identifier', 'SAON', 'PAON',
    'PPDCategory_Type', 'Record_Status - monthly_file_only', 'Street'
]
cols_to_keep = [col for col in df.columns if col not in columns_to_exclude]
df_covid = df[cols_to_keep].copy()

# Drop rows where the postcode is missing
df_covid.dropna(subset=["postcode"], inplace=True)

# Extract date components from 'Date_of_Transfer'
df_covid["year"] = pd.DatetimeIndex(df_covid['Date_of_Transfer']).year
df_covid["month"] = pd.DatetimeIndex(df_covid['Date_of_Transfer']).month
df_covid["week"] = df_covid['Date_of_Transfer'].dt.isocalendar().week  # ISO week (Monday as first day)
df_covid["day"] = pd.DatetimeIndex(df_covid['Date_of_Transfer']).day

# Split the postcode into two parts
df_covid[['postcodeA', 'postcodeB']] = df_covid['postcode'].str.split(expand=True)

# Filter out records with a property type of 'O'
df_covid = df_covid[df_covid['Property_Type'] != 'O'].copy()

# Create a Year-Month string column and convert it to datetime
df_covid['year_str'] = df_covid['year'].astype(str)
df_covid['month_str'] = df_covid['month'].astype(str)
df_covid["y_m"] = df_covid[['year_str', 'month_str']].apply(lambda x: "-".join(x), axis=1)
df_covid["y_m2"] = pd.to_datetime(df_covid["y_m"], infer_datetime_format=True)

# Revert any unintended type changes (year and month should remain integers)
df_covid['year'] = df_covid['year'].astype(int)
df_covid['month'] = df_covid['month'].astype(int)

# Define the county of interest
county = "GREATER LONDON"

# Determine the latest transfer date and compute the date a year ago
latest_transfer_date = df_covid['Date_of_Transfer'].max()
print("Latest transfer date:", latest_transfer_date)
covid_start = 2020
print("Covid start year is:", covid_start)

# Define the year range (using the max year in the data)
max_year = df_covid['year'].max()
print("Max year:", max_year)

# ------------------------------------------------------------------------
# keep london data since 2020
# ------------------------------------------------------------------------

# Filter data for records in Greater London within the selected year range
df_covid = df_covid[(df_covid["County"] == county) & (df_covid["year"] >= covid_start)]

# Extract unique values for districts, years, and year-month combinations
df_covid_districts = sorted(df_covid["District"].unique())
df_covid_years = sorted(df_covid["year"].unique())
df_covid_year_months = sorted(df_covid["y_m"].unique())

# Print the latest date and month from the data
latest_date = latest_transfer_date.date()
print("Latest date:", latest_date)
latest_month = latest_date.month
print("Latest month:", latest_month)

# Get and print the current date
current_date_str = datetime.now().strftime("%Y-%m-%d")
print("Current date:", current_date_str)

# 12 mins

Refreshing dataset by downloading the latest version...
Dataset URL: https://www.kaggle.com/datasets/lorentzyeung/price-paid-data-202304
License(s): other
100%|███████████████████████████████████████▉| 833M/834M [00:32<00:00, 29.0MB/s]
100%|████████████████████████████████████████| 834M/834M [00:32<00:00, 27.3MB/s]
Dataset refreshed.
Loading data from server...
Data loaded. Total records: 29853457
Sample data:
   price Date_of_Transfer  postcode Property_Type Old/New Duration PAON  \
0  42000       1995-12-21   NE4 9DN             S       N        F    8   
1  95000       1995-03-03  RM16 4UR             S       N        F   30   
2  74950       1995-10-03  CW10 9ES             D       Y        F   15   
3  43500       1995-11-14  TS23 3LA             S       N        F   19   
4  63000       1995-09-08  CA25 5QH             S       N        F    8   

         SAON            Street      Locality            Town/City  \
0  Unnamed: 8      MATFEN PLACE        FENHAM  NEWCASTLE UPON TYN

<ipython-input-3-3904edf83462>:81: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



Latest transfer date: 2024-12-30 00:00:00
Covid start year is: 2020
Max year: 2024
Latest date: 2024-12-30
Latest month: 12
Current date: 2025-02-24


# 1. Property Quantity Sold Monthly

In [4]:
# ---------------------------------------------------
# 2. Define Function for Timeline Plot

def timeline_count(df):
    print("\n[Step] Preparing timeline aggregator...")

    # Assume df is already pre-processed (df_covid) for Greater London since 2020.
    # Group by the pre-computed month (y_m2) to count transactions.
    grouped = df.groupby("y_m2")["Date_of_Transfer"].count().reset_index(name="Count")
    
    # Determine the latest transaction date and set chart end 10 days later.
    latest_date = df["Date_of_Transfer"].max()
    chart_end = latest_date + timedelta(days=10)
    
    # Build a monthly date range from Jan 2020 to chart_end using month-end frequency.
    date_range = pd.date_range("2020-01-01", chart_end, freq="ME")
    timeline_df = pd.DataFrame({"Dates": date_range})
    timeline_df["year"] = timeline_df["Dates"].dt.year
    timeline_df["month"] = timeline_df["Dates"].dt.month

    # Extract year and month from grouped y_m2
    grouped["year"] = grouped["y_m2"].dt.year
    grouped["month"] = grouped["y_m2"].dt.month

    # Merge monthly counts onto timeline_df; missing months get a count of 0.
    merged = pd.merge(
        timeline_df, 
        grouped[["year", "month", "Count"]], 
        on=["year", "month"], 
        how="left"
    ).fillna({"Count": 0})
    merged.sort_values("Dates", inplace=True)
    merged.reset_index(drop=True, inplace=True)

    # Create a mapping from (year, month) to (date, count)
    year_month_map = {
        (row["Dates"].year, row["Dates"].month): (row["Dates"], row["Count"])
        for _, row in merged.iterrows()
    }
    
    # Define key annotations for the timeline.
    annotation_specs = [
        {
            "year": 2021, 
            "month": 1, 
            "text": "BNO VISA<br>Launched<br>(Jan 31, 2021)", 
            "ax": 0, 
            "ay": -50
        },
        {
            "year": 2021, 
            "month": 2, 
            "text": "BNO VISA<br>Online Application<br>(Feb 23, 2021)", 
            "ax": 0, 
            "ay": 50
        },
        {
            "year": 2021, 
            "month": 3, 
            "text": "Stamp Duty<br>Increases in April<br>(Mar 01, 2021)", 
            "ax": 0, 
            "ay": -50
        },
        {
            "year": 2021, 
            "month": 5, 
            "text": "Stage 3<br>Easing<br>(May 17, 2021)", 
            "ax": 0, 
            "ay": 100
        },
        {
            "year": 2021, 
            "month": 6, 
            "text": "Stage 4<br>Easing Postponed<br>(Jun 21, 2021)", 
            "ax": 0, 
            "ay": -50
        },
        {
            "year": 2021, 
            "month": 8, 
            "text": "Scotland Removes<br>Most Lockdown<br>Measures<br>(Aug 09, 2021)", 
            "ax": 0, 
            "ay": 50
        },
        {
            "year": 2021, 
            "month": 11, 
            "text": "WHO<br>Announces<br>Omicron<br>(Nov 27, 2021)", 
            "ax": 0, 
            "ay": -90
        }
    ]
    
    # Build annotations from the mapping.
    annotations_list = []
    for spec in annotation_specs:
        if (spec["year"], spec["month"]) in year_month_map:
            x_date, y_count = year_month_map[(spec["year"], spec["month"])]
        else:
            x_date, y_count = (datetime(spec["year"], spec["month"], 28), 0)
        annotations_list.append(
            dict(
                x=x_date,
                y=y_count,
                xref="x",
                yref="y",
                text=spec["text"],
                showarrow=True,
                arrowhead=7,
                ax=spec["ax"],
                ay=spec["ay"]
            )
        )
    
    # Use the dataset's built-in year and month for the title.
    to_year = df["year"].max()
    to_month = df["month"].max()

    title_str = (
        f"Monthly Property Transaction Timeline from 2020 to {to_year}-{to_month}\n"
        f"(Last update in the database: {latest_date.date()})"
    )
    
    trace_time = go.Scatter(
        x=merged["Dates"],
        y=merged["Count"],
        name="Number of Sales",
        line=dict(color="#7F7F7F"),
        opacity=0.8
    )
    
    layout_timeline = go.Layout(
        title=title_str,
        xaxis=dict(
            rangeslider=dict(),
            type="date"
        ),
        annotations=annotations_list,
        autosize=False,
        width=1000,
        height=500
    )
    fig = dict(data=[trace_time], layout=layout_timeline)
    
    print("\nDisplaying interactive plot...")
    iplot(fig)
    
    print("\nSaving timeline plot to PNG file...")
    pio.write_image(fig, "./major_report_output/10_timeline_count_eng.png")
    print("Timeline plot saved to './major_report_output/10_timeline_count_eng.png'.")

# ---------------------------------------------------
# 3. Call the timeline_count function
print("\nCalling timeline_count() function...")
timeline_count(df_covid)
print("All steps completed.")



Calling timeline_count() function...

[Step] Preparing timeline aggregator...

Displaying interactive plot...



Saving timeline plot to PNG file...
Timeline plot saved to './major_report_output/10_timeline_count_eng.png'.
All steps completed.


In [5]:
# ---------------------------------------------------
# 2. Define Function for Timeline Plot

def timeline_count(df):
    print("\n[Step] Preparing timeline aggregator...")

    # Assume df is already pre-processed (df_covid) for Greater London since 2020.
    # Group by the pre-computed month (y_m2) to count transactions.
    grouped = df.groupby("y_m2")["Date_of_Transfer"].count().reset_index(name="Count")
    
    # Determine the latest transaction date and set chart end 10 days later.
    latest_date = df["Date_of_Transfer"].max()
    chart_end = latest_date + timedelta(days=10)
    
    # Build a monthly date range from Jan 2020 to chart_end using month-end frequency.
    date_range = pd.date_range("2020-01-01", chart_end, freq="ME")
    timeline_df = pd.DataFrame({"Dates": date_range})
    timeline_df["year"] = timeline_df["Dates"].dt.year
    timeline_df["month"] = timeline_df["Dates"].dt.month

    # Extract year and month from grouped y_m2
    grouped["year"] = grouped["y_m2"].dt.year
    grouped["month"] = grouped["y_m2"].dt.month

    # Merge monthly counts onto timeline_df; missing months get a count of 0.
    merged = pd.merge(
        timeline_df, 
        grouped[["year", "month", "Count"]], 
        on=["year", "month"], 
        how="left"
    ).fillna({"Count": 0})
    merged.sort_values("Dates", inplace=True)
    merged.reset_index(drop=True, inplace=True)

    # Create a mapping from (year, month) to (date, count)
    year_month_map = {
        (row["Dates"].year, row["Dates"].month): (row["Dates"], row["Count"])
        for _, row in merged.iterrows()
    }
    
    # Define key annotations for the timeline.
    annotation_specs = [
        {
            "year": 2021, 
            "month": 1, 
            "text": "BNO VISA<br>開放申請<br>(Jan 31, 2021)", 
            "ax": 0, 
            "ay": -50
        },
        {
            "year": 2021, 
            "month": 2, 
            "text": "BNO VISA<br>開放網上申請<br>(Feb 23, 2021)", 
            "ax": 0, 
            "ay": 50
        },
        {
            "year": 2021, 
            "month": 3, 
            "text": "印花稅<br>4月上漲<br>(Mar 01, 2021)", 
            "ax": 0, 
            "ay": -50
        },
        {
            "year": 2021, 
            "month": 5, 
            "text": "第3階段<br>放寬<br>(May 17, 2021)", 
            "ax": 0, 
            "ay": 100
        },
        {
            "year": 2021, 
            "month": 6, 
            "text": "原定6月21日<br>第4階段放寬延期<br>(Jun 21, 2021)", 
            "ax": 0, 
            "ay": -50
        },
        {
            "year": 2021, 
            "month": 8, 
            "text": "蘇格蘭移除<br>大部份肺炎<br>相關封鎖措施<br>(Aug 09, 2021)", 
            "ax": 0, 
            "ay": 50
        },
        {
            "year": 2021, 
            "month": 11, 
            "text": "世衛<br>發表<br>Omicron<br>(Nov 27, 2021)", 
            "ax": 0, 
            "ay": -90
        }
    ]
    
    # Build annotations from the mapping.
    annotations_list = []
    for spec in annotation_specs:
        if (spec["year"], spec["month"]) in year_month_map:
            x_date, y_count = year_month_map[(spec["year"], spec["month"])]
        else:
            x_date, y_count = (datetime(spec["year"], spec["month"], 28), 0)
        annotations_list.append(
            dict(
                x=x_date,
                y=y_count,
                xref="x",
                yref="y",
                text=spec["text"],
                showarrow=True,
                arrowhead=7,
                ax=spec["ax"],
                ay=spec["ay"]
            )
        )
    
    # Use the dataset's built-in year and month for the title.
    to_year = df["year"].max()
    to_month = df["month"].max()

    title_str = (
        f"2020年至{to_year}年{to_month}月倫敦房屋成交數量時間軸\n"
        f"(所使用數據庫最後更新至{latest_date.date()})"
    )
    
    trace_time = go.Scatter(
        x=merged["Dates"],
        y=merged["Count"],
        name="Number of Sales",
        line=dict(color="#7F7F7F"),
        opacity=0.8
    )
    
    layout_timeline = go.Layout(
        title=title_str,
        xaxis=dict(
            rangeslider=dict(),
            type="date"
        ),
        annotations=annotations_list,
        autosize=False,
        width=1000,
        height=500
    )
    fig = dict(data=[trace_time], layout=layout_timeline)
    
    print("\nDisplaying interactive plot...")
    iplot(fig)
    
    print("\nSaving timeline plot to PNG file...")
    pio.write_image(fig, "./major_report_output/10_timeline_count_chi.png")
    print("Timeline plot saved to './major_report_output/10_timeline_count_chi.png'.")

# ---------------------------------------------------
# 3. Call the timeline_count function
print("\nCalling timeline_count() function...")
timeline_count(df_covid)
print("All steps completed.")



Calling timeline_count() function...

[Step] Preparing timeline aggregator...

Displaying interactive plot...



Saving timeline plot to PNG file...
Timeline plot saved to './major_report_output/10_timeline_count_chi.png'.
All steps completed.


# 2. Property Price Sold Monthly

In [6]:
# -----------------------------
# 2. Define Function for Monthly Median Price Timeline Plot
def timeline_median(df):
    print("\n[Step] Preparing median price timeline aggregator...")
    
    # Group by the pre-computed year-month (y_m2) to compute the monthly median price
    grouped = df.groupby("y_m2")["price"].median().reset_index(name="MedianPrice")
    # Extract year and month from the grouped datetime
    grouped["year"] = grouped["y_m2"].dt.year
    grouped["month"] = grouped["y_m2"].dt.month
    
    # Determine the latest transaction date and define chart_end (10 days later)
    latest_date = df["Date_of_Transfer"].max()
    chart_end = latest_date + timedelta(days=10)
    
    # Build a complete monthly date range from Jan 2020 to chart_end (month-end frequency)
    date_range = pd.date_range("2020-01-01", chart_end, freq="ME")
    timeline_df = pd.DataFrame({"Dates": date_range})
    timeline_df["year"] = timeline_df["Dates"].dt.year
    timeline_df["month"] = timeline_df["Dates"].dt.month
    
    # Merge the grouped median prices onto the complete monthly timeline; fill missing with 0
    merged = pd.merge(
        timeline_df, 
        grouped[["year", "month", "MedianPrice"]],
        on=["year", "month"],
        how="left"
    )
    merged["MedianPrice"] = merged["MedianPrice"].fillna(0)
    merged.sort_values("Dates", inplace=True)
    merged.reset_index(drop=True, inplace=True)
    
    # Build a mapping from (year, month) to (Dates, MedianPrice)
    year_month_map = {
        (row["Dates"].year, row["Dates"].month): (row["Dates"], row["MedianPrice"])
        for _, row in merged.iterrows()
    }
    
    # Define key event annotations (as before)
    annotation_specs = [
        {
            "year": 2021, 
            "month": 1, 
            "text": "BNO VISA<br>Launched<br>(Jan 31, 2021)", 
            "ax": 0, 
            "ay": -30
        },
        {
            "year": 2021, 
            "month": 2, 
            "text": "BNO VISA<br>Online Application<br>(Feb 23, 2021)", 
            "ax": 0, 
            "ay": 50
        },
        {
            "year": 2021, 
            "month": 3, 
            "text": "Stamp Duty<br>Increases in April<br>(Mar 01, 2021)", 
            "ax": 0, 
            "ay": -60
        },
        {
            "year": 2021, 
            "month": 5, 
            "text": "Stage 3<br>Easing<br>(May 17, 2021)", 
            "ax": 0, 
            "ay": 100
        },
        {
            "year": 2021, 
            "month": 6, 
            "text": "Stage 4<br>Easing Postponed<br>(Jun 21, 2021)", 
            "ax": 0, 
            "ay": -100
        },
        {
            "year": 2021, 
            "month": 8, 
            "text": "Scotland Removes<br>Most Lockdown<br>Measures<br>(Aug 09, 2021)", 
            "ax": 0, 
            "ay": 50
        },
        {
            "year": 2021, 
            "month": 11, 
            "text": "WHO<br>Announces<br>Omicron<br>(Nov 27, 2021)", 
            "ax": 0, 
            "ay": -90
        }
    ]
    
    annotations_list = []
    for spec in annotation_specs:
        if (spec["year"], spec["month"]) in year_month_map:
            x_date, y_val = year_month_map[(spec["year"], spec["month"])]
        else:
            x_date, y_val = (datetime(spec["year"], spec["month"], 28), 0)
        annotations_list.append(dict(
            x=x_date,
            y=y_val,
            xref="x",
            yref="y",
            text=spec["text"],
            showarrow=True,
            arrowhead=7,
            ax=spec["ax"],
            ay=spec["ay"]
        ))
    
    # Title for the timeline plot
    to_year = df["year"].max()
    to_month = df["month"].max()
    title_str = (
        f"Monthly Median Housing Transaction Price Timeline from 2020 to {to_year}-{to_month}\n"
        f"(Last update: {latest_date.date()})"
    )
    
    # Build the Plotly trace and layout
    trace_median = go.Scatter(
        x=merged["Dates"],
        y=merged["MedianPrice"],
        name="Median Price",
        line=dict(color="#7F7F7F"),
        opacity=0.8
    )
    
    layout = go.Layout(
        title=title_str,
        xaxis=dict(
            rangeslider=dict(),
            type="date"
        ),
        annotations=annotations_list,
        autosize=False,
        width=1000,
        height=500
    )
    
    fig = go.Figure(data=[trace_median], layout=layout)
    
    print("\nDisplaying interactive median price timeline plot...")
    fig.show()
    
    print("\nSaving median price timeline plot to PNG file...")
    pio.write_image(fig, "./major_report_output/10_timeline_median_eng.png")
    print("Median price timeline plot saved to './major_report_output/10_timeline_median_eng.png'.")

# -----------------------------
# 3. Call the timeline_median function
print("\nCalling timeline_median() function...")
timeline_median(df_covid)
print("All steps completed for monthly median prices.")



Calling timeline_median() function...

[Step] Preparing median price timeline aggregator...

Displaying interactive median price timeline plot...



Saving median price timeline plot to PNG file...
Median price timeline plot saved to './major_report_output/10_timeline_median_eng.png'.
All steps completed for monthly median prices.


In [7]:
# -----------------------------
# 2. Define Function for Monthly Median Price Timeline Plot
def timeline_median(df):
    print("\n[Step] Preparing median price timeline aggregator...")
    
    # Group by the pre-computed year-month (y_m2) to compute the monthly median price
    grouped = df.groupby("y_m2")["price"].median().reset_index(name="MedianPrice")
    # Extract year and month from the grouped datetime
    grouped["year"] = grouped["y_m2"].dt.year
    grouped["month"] = grouped["y_m2"].dt.month
    
    # Determine the latest transaction date and define chart_end (10 days later)
    latest_date = df["Date_of_Transfer"].max()
    chart_end = latest_date + timedelta(days=10)
    
    # Build a complete monthly date range from Jan 2020 to chart_end (month-end frequency)
    date_range = pd.date_range("2020-01-01", chart_end, freq="ME")
    timeline_df = pd.DataFrame({"Dates": date_range})
    timeline_df["year"] = timeline_df["Dates"].dt.year
    timeline_df["month"] = timeline_df["Dates"].dt.month
    
    # Merge the grouped median prices onto the complete monthly timeline; fill missing with 0
    merged = pd.merge(
        timeline_df, 
        grouped[["year", "month", "MedianPrice"]],
        on=["year", "month"],
        how="left"
    )
    merged["MedianPrice"] = merged["MedianPrice"].fillna(0)
    merged.sort_values("Dates", inplace=True)
    merged.reset_index(drop=True, inplace=True)
    
    # Build a mapping from (year, month) to (Dates, MedianPrice)
    year_month_map = {
        (row["Dates"].year, row["Dates"].month): (row["Dates"], row["MedianPrice"])
        for _, row in merged.iterrows()
    }
    
    # Define key event annotations (as before)
    annotation_specs = [
        {"year": 2021, "month": 1, "text": "BNO VISA<br>開放申請<br>(Jan 31, 2021)", "ax": 0, "ay": -30},
        {"year": 2021, "month": 2, "text": "BNO VISA<br>開放網上申請<br>(Feb 23, 2021)", "ax": 0, "ay": 50},
        {"year": 2021, "month": 3, "text": "印花稅<br>4月上漲<br>(Mar 01, 2021)", "ax": 0, "ay": -60},
        {"year": 2021, "month": 5, "text": "第3階段放寬<br>預計6月<br>第4階段放寬<br>(May 17, 2021)", "ax": 0, "ay": 100},
        {"year": 2021, "month": 6, "text": "原定6月21日<br>第4階段放寬延期<br>(Jun 21, 2021)", "ax": 0, "ay": -100},
        {"year": 2021, "month": 8, "text": "蘇格蘭移除<br>大部份肺炎相關<br>封鎖措施<br>(Aug 09, 2021)", "ax": 0, "ay": 50},
        {"year": 2021, "month": 11, "text": "世衛<br>發表<br>Omicron<br>(Nov 27, 2021)", "ax": 0, "ay": -90}
    ]
    
    annotations_list = []
    for spec in annotation_specs:
        if (spec["year"], spec["month"]) in year_month_map:
            x_date, y_val = year_month_map[(spec["year"], spec["month"])]
        else:
            x_date, y_val = (datetime(spec["year"], spec["month"], 28), 0)
        annotations_list.append(dict(
            x=x_date,
            y=y_val,
            xref="x",
            yref="y",
            text=spec["text"],
            showarrow=True,
            arrowhead=7,
            ax=spec["ax"],
            ay=spec["ay"]
        ))
    
    # Title for the timeline plot
    to_year = df["year"].max()
    to_month = df["month"].max()
    title_str = (
        f'2020年至{to_year}年{to_month}月倫敦房屋銷售成交價中位數時間軸'
        # f"Monthly Median Housing Transaction Price Timeline from 2020 to {to_year}-{to_month}\n"
        f"(所使用數據庫最後更新至{latest_date.date()})"
    )
    
    # Build the Plotly trace and layout
    trace_median = go.Scatter(
        x=merged["Dates"],
        y=merged["MedianPrice"],
        name="Median Price",
        line=dict(color="#7F7F7F"),
        opacity=0.8
    )
    
    layout = go.Layout(
        title=title_str,
        xaxis=dict(
            rangeslider=dict(),
            type="date"
        ),
        annotations=annotations_list,
        autosize=False,
        width=1000,
        height=500
    )
    
    fig = go.Figure(data=[trace_median], layout=layout)
    
    print("\nDisplaying interactive median price timeline plot...")
    fig.show()
    
    print("\nSaving median price timeline plot to PNG file...")
    pio.write_image(fig, "./major_report_output/10_timeline_median_chi.png")
    print("Median price timeline plot saved to './major_report_output/10_timeline_median_chi.png'.")

# -----------------------------
# 3. Call the timeline_median function
print("\nCalling timeline_median() function...")
timeline_median(df_covid)
print("All steps completed for monthly median prices.")



Calling timeline_median() function...

[Step] Preparing median price timeline aggregator...

Displaying interactive median price timeline plot...



Saving median price timeline plot to PNG file...
Median price timeline plot saved to './major_report_output/10_timeline_median_chi.png'.
All steps completed for monthly median prices.


# 3. Moving Average

In [8]:
# -----------------------------
# 2. Define Function for Momentum Analysis (Plotly Version)
def momentum_analysis_plotly(df):
    """
    Plot London's housing transaction momentum using daily transaction counts
    and multiple rolling averages. Assumes df (e.g. df_covid) is already preprocessed
    (filtered for Greater London from 2020 onward and with proper date columns).
    """
    # Filter data for 2020 onward (if not already done)
    temp = df.copy()
    
    # Group by Date_of_Transfer to calculate daily transaction count
    average = temp.groupby("Date_of_Transfer").agg({"price": "count"}).reset_index()
    average.columns = ["date", "count"]
    
    # Calculate rolling averages on the daily transaction count
    average["30-day MA"] = average["count"].rolling(30).mean()
    average["50-day MA"] = average["count"].rolling(50).mean()
    average["90-day MA"] = average["count"].rolling(90).mean()
    average["200-day MA"] = average["count"].rolling(200).mean()
    
    latest_date = temp["Date_of_Transfer"].max()
    
    # Create Plotly traces for each rolling average
    trace_30 = go.Scatter(
        x = average["date"],
        y = average["30-day MA"],
        mode = 'lines',
        name = "30-day MA (Reference)",
        line = dict(color = 'blue')
    )
    trace_50 = go.Scatter(
        x = average["date"],
        y = average["50-day MA"],
        mode = 'lines',
        name = "50-day MA",
        line = dict(color = 'orange')
    )
    trace_90 = go.Scatter(
        x = average["date"],
        y = average["90-day MA"],
        mode = 'lines',
        name = "90-day MA",
        line = dict(color = 'green')
    )
    trace_200 = go.Scatter(
        x = average["date"],
        y = average["200-day MA"],
        mode = 'lines',
        name = "200-day MA",
        line = dict(color = 'red')
    )
    
    data = [trace_30, trace_50, trace_90, trace_200]
    
    title_str = f"London Property Sales Momentum Analysis\n(Data last updated: {latest_date.date()})"
    layout = go.Layout(
        title = title_str,
        xaxis = dict(title="Date"),
        yaxis = dict(title="Daily Transaction Count (Rolling Average)"),
        autosize = False,
        width = 1200,  # Wider width for the graph
        height = 700,
        # Place the legend inside the graph (top-right)
        legend = dict(
            x = 0.95,
            y = 0.95,
            xanchor = "right",
            yanchor = "top",
            bgcolor = 'rgba(255,255,255,0.5)'
        )
    )
    
    fig = go.Figure(data=data, layout=layout)
    fig.show()
    
    output_file = "./major_report_output/11_momentum_chi.png"
    print(f"[Momentum] Saving momentum analysis plot to {output_file} ...")
    pio.write_image(fig, output_file, format='png', scale=2)
    print("[Momentum] Momentum chi analysis plot saved successfully.")

# -----------------------------
# 3. Call the momentum_analysis_plotly function
print("\nCalling momentum_analysis_plotly() function...")
momentum_analysis_plotly(df_covid)
print("Momentum analysis completed.")



Calling momentum_analysis_plotly() function...


[Momentum] Saving momentum analysis plot to ./major_report_output/11_momentum_chi.png ...
[Momentum] Momentum chi analysis plot saved successfully.
Momentum analysis completed.


In [9]:
# -----------------------------
# 2. Define Function for Momentum Analysis (Plotly Version)
def momentum_analysis_plotly(df):
    """
    Plot London's housing transaction momentum using daily transaction counts
    and multiple rolling averages. Assumes df (e.g. df_covid) is already preprocessed
    (filtered for Greater London from 2020 onward and with proper date columns).
    """
    # Filter data for 2020 onward (if not already done)
    temp = df.copy()
    
    # Group by Date_of_Transfer to calculate daily transaction count
    average = temp.groupby("Date_of_Transfer").agg({"price": "count"}).reset_index()
    average.columns = ["date", "count"]
    
    # Calculate rolling averages on the daily transaction count
    average["30-day MA"] = average["count"].rolling(30).mean()
    average["50-day MA"] = average["count"].rolling(50).mean()
    average["90-day MA"] = average["count"].rolling(90).mean()
    average["200-day MA"] = average["count"].rolling(200).mean()
    
    latest_date = temp["Date_of_Transfer"].max()
    
    # Create Plotly traces for each rolling average
    trace_30 = go.Scatter(
        x=average["date"],
        y=average["30-day MA"],
        mode='lines',
        name="30天移動平均（參考）",
        line=dict(color='blue')
    )
    trace_50 = go.Scatter(
        x=average["date"],
        y=average["50-day MA"],
        mode='lines',
        name="50天移動平均",
        line=dict(color='orange')
    )
    trace_90 = go.Scatter(
        x=average["date"],
        y=average["90-day MA"],
        mode='lines',
        name="90天移動平均",
        line=dict(color='green')
    )
    trace_200 = go.Scatter(
        x=average["date"],
        y=average["200-day MA"],
        mode='lines',
        name="200天移動平均",
        line=dict(color='red')
    )
    
    data = [trace_30, trace_50, trace_90, trace_200]
    
    title_str = f"倫敦房產銷售動力分析\n（數據最新更新：{latest_date.date()}）"
    layout = go.Layout(
        title=title_str,
        xaxis=dict(title="日期"),
        yaxis=dict(title="每日交易數量（移動平均）"),
        autosize=False,
        width=1200,
        height=700,
        legend=dict(
            x=0.95,
            y=0.95,
            xanchor="right",
            yanchor="top",
            bgcolor='rgba(255,255,255,0.5)'
        )
    )
    
    fig = go.Figure(data=data, layout=layout)
    fig.show()
    
    output_file = "./major_report_output/11_momentum_eng.png"
    print(f"[Momentum] Saving momentum analysis plot to {output_file} ...")
    pio.write_image(fig, output_file, format='png', scale=2)
    print("[Momentum] Momentum eng analysis plot saved successfully.")

# -----------------------------
# 3. Call the momentum_analysis_plotly function
print("\nCalling momentum_analysis_plotly() function...")
momentum_analysis_plotly(df_covid)
print("Momentum analysis completed.")



Calling momentum_analysis_plotly() function...


[Momentum] Saving momentum analysis plot to ./major_report_output/11_momentum_eng.png ...
[Momentum] Momentum eng analysis plot saved successfully.
Momentum analysis completed.


# PDF Generation

In [10]:
import os
from fpdf import FPDF
from datetime import datetime

# Ensure the output directory exists
os.makedirs("./output", exist_ok=True)

# Report details
report_title = "Automated UK Property Price Analysis Report"
report_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
file_date = datetime.now().strftime("%Y-%m-%d")
developer_info = "Developed by AI Developer: Lorentz Yeung"  

# Additional introductory text for the cover page
intro_text = (
    "This report is an excerpt from our fully automated pipeline that monitors, processes, "
    "and analyzes the UK Government's Price Paid Data on a monthly basis. The pipeline automatically "
    "scrapes data from the official webpage, performs data engineering, and generates up-to-date visualizations. "
    "The charts presented herein represent a demo of the entire process, which is updated automatically each month."
)

# Create a landscape PDF (A4 size)
pdf = FPDF(orientation="L", unit="mm", format="A4")
pdf.set_auto_page_break(auto=True, margin=15)

# --- Cover Page (First Page) ---
pdf.add_page()

# Title
pdf.set_font("Arial", "B", 28)
pdf.cell(0, 20, report_title, ln=True, align="C")
pdf.ln(10)

# Report creation date
pdf.set_font("Arial", "", 16)
pdf.cell(0, 10, f"Report Creation Date: {report_date}", ln=True, align="C")
pdf.ln(5)

# Developer information
pdf.cell(0, 10, developer_info, ln=True, align="C")
pdf.ln(20)

# Introductory text
pdf.set_font("Arial", "", 14)
pdf.multi_cell(0, 10, intro_text, align="C")

# Path to images
image_dir = "./major_report_output"
image_files = [
    "10_timeline_count_eng.png", "10_timeline_count_chi.png",
    "10_timeline_median_eng.png", "10_timeline_median_chi.png",
    "11_momentum_eng.png", "11_momentum_chi.png"
]

# --- Add Images to PDF ---
for img in image_files:
    img_path = os.path.join(image_dir, img)
    if os.path.exists(img_path):
        pdf.add_page()
        pdf.image(img_path, x=10, y=10, w=277)  # Centered in landscape A4 (297mm width)

# --- Final Blank Page ---
pdf.add_page()
pdf.set_font("Arial", "B", 24)
pdf.cell(0, 20, "This is the end of the report.", ln=True, align="C")

# Save the PDF
output_pdf_path = f"./output/mini_report_{file_date}.pdf"
pdf.output(output_pdf_path, "F")

print(f"Full PDF report created: {output_pdf_path}")


Full PDF report created: ./output/mini_report_2025-02-24.pdf
